In [94]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
import pandas as pd
import matplotlib.pyplot as plt

%matplotlib inline

In [7]:
# Directory setup
BASE_DIR = '..'

TFRECORDS_DIR = '../data/lsms_tfrecords/'
LSMS_CSV_PATH = '../data/lsms_clusters.csv'

In [6]:
df_lsms = pd.read_csv(LSMS_CSV_PATH)
display(df_lsms)

,country,hh_size,consumption,density,lat,lon
0,ethiopia,1,7.114359,0,13.522697,39.440907
1,ethiopia,6,6.933497,1,9.602403,41.866592
2,ethiopia,2,7.354854,1,11.294477,36.448727
3,ethiopia,5,1.129382,1,9.503581,42.070496
4,ethiopia,4,1.924262,1,8.970294,36.525711
...,...,...,...,...,...,...
6508,ethiopia,3,4.283961,1,9.600062,41.245560
6509,ethiopia,3,2.158788,0,9.062462,38.757240
6510,ethiopia,8,1.986287,0,10.821666,40.187645
6511,ethiopia,7,2.943372,1,11.800598,37.128471


In [ ]:
sample_tfrecord = TFRECORDS_DIR + 'ethiopia_00.tfrecord'
"""
Get feature map
Plot image using matplot lib
Create dataset using satellite imagery and consumption data
"""